In [ ]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Initialize TimeSeries with your Alpha Vantage API key
api_key = 'WBJH0PBX2P5QR14U'
ts = TimeSeries(key=api_key, output_format='pandas')

# Fetch historical data for Apple
symbol = 'AAPL'
data, meta_data = ts.get_daily(symbol=symbol, outputsize='full')
data.reset_index(inplace=True)

# Focus on the '4. close' prices
close_prices = data[['4. close']].values

# Normalize the close prices
scaler = MinMaxScaler(feature_range=(0, 1))
close_prices_scaled = scaler.fit_transform(close_prices)

# Function to create dataset
def create_dataset(dataset, look_back=60):
    X, Y = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

# Prepare the input features and labels
X, y = create_dataset(close_prices_scaled)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(LSTM(50))
model.add(Dense(1))

# Compile and train the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)
#save the model
import os
import joblib
save_dir = '/tmp'
os.makedirs(save_dir, exist_ok=True)
model.save(os.path.join(save_dir, 'my_model.keras'))
joblib.dump(scaler, os.path.join(save_dir, 'scaler.pkl'))
from sklearn.metrics import mean_squared_error

# Make predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Inverse transform predictions and actuals to original scale
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
y_train_inv = scaler.inverse_transform([y_train])
y_test_inv = scaler.inverse_transform([y_test])

# Calculate RMSE
train_score = np.sqrt(mean_squared_error(y_train_inv[0], train_predict[:,0]))
test_score = np.sqrt(mean_squared_error(y_test_inv[0], test_predict[:,0]))
print(f'Train Score: {train_score:.2f} RMSE')
print(f'Test Score: {test_score:.2f} RMSE')
#from sklearn.metrics import mean_squared_error

# Make predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Inverse transform predictions and actuals to original scale
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
y_train_inv = scaler.inverse_transform([y_train])
y_test_inv = scaler.inverse_transform([y_test])

# Calculate RMSE
train_score = np.sqrt(mean_squared_error(y_train_inv[0], train_predict[:,0]))
test_score = np.sqrt(mean_squared_error(y_test_inv[0], test_predict[:,0]))
print(f'Train Score: {train_score:.2f} RMSE')
print(f'Test Score: {test_score:.2f} RMSE')

# Train Score: 10.51 RMSE
# Test Score: 4.30 RMSE

In [ ]:
import matplotlib.pyplot as plt

# Plotting the results with enhanced visualization using subplots
fig, axs = plt.subplots(2, figsize=(14, 10))

# Plotting actual vs predicted prices for the entire dataset
axs[0].plot(actual_prices, color='darkblue', label='Actual AAPL Price')
axs[0].plot(predicted_prices, color='pink', label='Predicted AAPL Price')
axs[0].set_title('Apple Stock Price Prediction - Entire Dataset', fontsize=16)
axs[0].set_xlabel('Time', fontsize=14)
axs[0].set_ylabel('Apple Stock Price (USD)', fontsize=14)
axs[0].legend(loc='upper right', fontsize=12)
axs[0].grid(True, which='both', linestyle='--', linewidth=0.5)

# Plotting actual vs predicted prices for the testing dataset
test_indices = np.arange(len(actual_prices) - len(y_test), len(actual_prices))
axs[1].plot(test_indices, actual_prices[-len(y_test):], color='red', label='Actual AAPL Price')
axs[1].plot(test_indices, test_predict, color='royalblue', label='Predicted AAPL Price')
axs[1].set_title('Apple Stock Price Prediction - Test Dataset', fontsize=16)
axs[1].set_xlabel('Time', fontsize=14)
axs[1].set_ylabel('Apple Stock Price (USD)', fontsize=14)
axs[1].legend(loc='upper right', fontsize=12)
axs[1].grid(True, which='both', linestyle='--', linewidth=0.5)

# Adding annotations for RMSE
axs[0].annotate(f'Train RMSE: {train_score:.2f}', xy=(0.05, 0.95), xycoords='axes fraction', fontsize=12, backgroundcolor='white')
axs[1].annotate(f'Test RMSE: {test_score:.2f}', xy=(0.05, 0.95), xycoords='axes fraction', fontsize=12, backgroundcolor='white')

# Show the plot
plt.tight_layout()
plt.show()